# Downloading Dataset

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-datasets/122892/296485/celebrities-100k.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559398353&Signature=KOf7hc8aAQ71VzYxa7okNbGcOD8JnLF2pLj6ZKsO4o5aCLwY1COiFQ5nwWvKa%2ByXh%2BV39Xmm3ew4R8ZtsaVS4HfFbHyEpagAXWfm7GxPs9S3qMgKYByDJoR9uPaSqmakPDAhGBrGT5GZwE7YDIR2G5%2BbKrVC%2BcbWg9PJNDYE%2B%2BJiMZlGv%2BuAGo0%2BFkJjF%2FB5iXGJonlqljkTiRQJ8R89Ru0dz6LO1tKc8TrvSOEGBGAsNMV4GFoNbPzLssmAM7Rg1XGP271Q9rRqRuXhGRYDwEEHOdQNXlceSsLooi3tl%2B%2FpaPuKisk7vQ%2BUc0nC%2FuekVw4hPYqcqP4HE8%2BkG0NwGg%3D%3D" -O "celebrities-100k.zip" -c

In [0]:
!unzip 'celebrities-100k.zip'

In [0]:
!unzip '100k.zip'

# Building Model and Training

In [0]:
!mkdir 'Model Architecture'

In [0]:
from __future__ import print_function, division

from keras.layers import Dense, Dropout, Flatten, Reshape, Input, BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from PIL import Image
from keras.utils import plot_model

import matplotlib.pyplot as plt
import numpy as np
import sys
import os as os
import tensorflow as tf

In [0]:
noise_shape = 100
nh = 64
nw = 64
nc = 3
img_shape = (nh, nw, nc)

# Carefully chosen parameters 
d_opt = Adam(lr = 0.00004, beta_1 = 0.5)
g_opt = Adam(lr = 0.0002, beta_1 = 0.5)

In [0]:
def build_generator():
  
  model = Sequential()
  model.add(Dense(128 * 4 * 4, activation="relu", input_dim = noise_shape, name = 'Dense_Layer'))
  model.add(Reshape((4, 4, 128) , name = "Reshape_Layer"))
  
  model.add(UpSampling2D(name = 'UpSampling_Layer'))
  model.add(Conv2D(64, kernel_size=3,  padding="same", name = "Conv_Layer"))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU'))
  model.add(BatchNormalization(momentum=0.8, name = "Batch_Norm"))
  
  model.add(UpSampling2D(name = 'UpSampling_Layer_2'))
  model.add(Conv2D(32, kernel_size=3, padding="same", name = 'Conv_Layer_2'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_2'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_2'))
  
  model.add(UpSampling2D(name = 'UpSampling_Layer_3'))
  model.add(Conv2D(16, kernel_size=3, padding="same", name = 'Conv_Layer_3'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_3'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_3'))
  
  model.add(UpSampling2D(name = 'UpSampling_Layer_4'))
  model.add(Conv2D(8, kernel_size=3, padding="same", name = 'Conv_Layer_4'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_4'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_4'))
  
  model.add(Conv2D(4, kernel_size=3, padding="same", name = 'Conv_Layer_5'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_5'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_5'))
  
  model.add(Conv2D(3, kernel_size=3, padding="same", name = 'Conv_Layer_6'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_6'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_6'))
  model.add(Activation("tanh"))
   
  print(model.summary())
  
  #Picturizing model
  plot_model(model, to_file='Model Architecture/generator.png')
  
  noise = Input(shape = (noise_shape,))
  img = model(noise)
  
  return Model(inputs = noise, outputs = img, name = "Generator")
  

In [0]:
generator = build_generator()
generator.summary()

In [0]:
def build_discriminator():
  model = Sequential()
  
  model.add(Conv2D(4, kernel_size=3, input_shape = img_shape, padding="same", name = 'Conv_Layer'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm'))
  
  model.add(Conv2D(8, kernel_size=3, strides = 2, name = 'Conv_Layer_2'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_2'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_2'))
  
  model.add(Conv2D(16, kernel_size=3, strides = 2, name = 'Conv_Layer_3'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_3'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_3'))
  
  model.add(Conv2D(32, kernel_size=3, strides = 2, name = 'Conv_Layer_4'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_4'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_4'))
  
  model.add(Conv2D(64, kernel_size=2, strides = 1, name = 'Conv_Layer_5'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_5'))
  model.add(BatchNormalization(momentum=0.8, name = 'Batch_Norm_5'))
  
  
  model.add(Flatten(name = 'Flatten'))
  model.add(Dense(512, name = 'Dense_Layer'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_6'))
  
  model.add(Dense(256, name = 'Dense_Layer_2'))
  model.add(LeakyReLU(alpha=0.2, name = 'Leaky_ReLU_7'))
  model.add(Dense(1, activation='sigmoid', name = 'Dense_with_Sigmoid'))
  
  print(model.summary())
  
  #Picturizing model
  plot_model(model, to_file='Model Architecture/discriminator.png')
  
  img = Input(shape = img_shape)
  validity = model(img)
  
  return Model(inputs = img, outputs = validity, name = 'Discriminator')

In [0]:
discriminator = build_discriminator()
discriminator.summary()

discriminator.compile(loss = 'binary_crossentropy', optimizer = d_opt, metrics = ['accuracy'])

In [0]:
z = Input(shape = (noise_shape,), name = "Input_Noise")
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss = 'binary_crossentropy', optimizer = g_opt)
combined.summary()

#Picturizing model
plot_model(combined, to_file='Model Architecture/combined.png')

In [0]:
print(generator.summary())
print(discriminator.summary())
print(combined.summary())

In [0]:
def get_batch(width, height):
  image_files = os.listdir('/content/100k/')[:]
  data_batch = []
  
  for file in image_files:
    img = Image.open('/content/100k/'+file).resize([width, height])
    data_batch.append(np.array(img.convert('RGB')))
    
  data_batch = np.array(data_batch)
  print(data_batch.shape)
  return data_batch

In [0]:
def train(batch_size = 64, epochs = 4000):
  
  
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))
  
  X_train = get_batch(nw, nh)
  X_train = (X_train.astype(np.float32) - 127.5) / 127.5
  
  for epoch in range(epochs):
    
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, noise_shape))
    
    gen_imgs = generator.predict(noise)

    #Training Discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

    #Training Generator
    g_loss = combined.train_on_batch(noise, valid)

    #Progress
    print("epoch: " + str(epoch+1) + " " + "D_Loss = " + str(d_loss[0]) + " " + "acc: " + str(d_loss[1]*100) + " " +  "G_Loss = " + str(g_loss))

In [0]:
train(64, 90000)

In [0]:
!mkdir 'images'

# Generating Images

In [0]:
def save_imgs(epoch):
  r, c = 3, 3
  noise = np.random.normal(0, 1, (r * c, noise_shape))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = (1/2.5) * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,:])
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig("images/%d.png" % epoch)
  plt.close()

In [0]:
save_imgs(100000)